# Decorator

In [29]:
# without any augment
# return original function return value
# @Property
def NoAugmentDecoratorReturnFunctionForm(func):    
    
    # some kinds of pre-processes..
    return func()
    
    # some kinds of post-processes..
    
# -> NoAugmentDecoratorReturnFunctionForm(func())
@NoAugmentDecoratorReturnFunctionForm
def NoAugmentFunc():
    print("hello World!")
    
NoAugmentFunc

hello World!


In [30]:
# without any augment
# return original function
def NoAugmentDecorator(func):
    return func

@NoAugmentDecorator
def NoAugmentFunc():
    print("hello world!")

# (NoAugmentDecorator(func))()
NoAugmentFunc()

hello world!


In [31]:
# without any augment
# return modified function
def NoAugmentUpperDecorator(func):
    
    def wrapper():
        return func().upper()
    return wrapper

@NoAugmentUpperDecorator
def NoAugmentFunc():
    return 'hello world!'

# (NoAugmentUpperDecorator(func))() -> wrapper() -> func().upper()
NoAugmentFunc()

'HELLO WORLD!'

In [35]:
# without any augment
# return original function with augment
def NoAugmentDecorator(func):
    
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@NoAugmentDecorator
def Func(a,b,c):
    print(a,b,c)

# (NoAugmentDecorator(func))(*args, **kwargs)
Func(1,2,3)

1 2 3


In [39]:
# with augment
# return original function with augment
def Decorator(*args, **kwargs):
    print("decorator augment : ", args, kwargs)
    
    def func_wrapper(func):
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)
        return wrapper
    return func_wrapper

@Decorator(1, k=2)
def Func(a,b,c):
    print(a,b,c)

# ((Decorator(*args, **kwargs))(func))(*args, **kwargs)
Func(1,2,3)

decorator augment :  (1,) {'k': 2}
1 2 3


In [46]:
# error handling with decorator

def ValueErrorHandler(a=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except ValueError as e:
                print(e)
        return wrapper
    return decorator

# (ValueErrorHandler(func))(*arg, **kwargs)
@ValueErrorHandler()
def func(a,b,c):
    
    print(a,b,c)
    
    if a > 0:
        raise ValueError("ValueError")

func(1,2,3)

1 2 3
ValueError


In [52]:
def ErrorHandler(error = Exception):
    def decorator(func):
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except error as e:
                print(e, e.__class__.__name__)
        return wrapper
    return decorator

@ErrorHandler(error = (TypeError, ValueError))
def func(a,b,c):
    
    print(a,b,c)
    
    if a > 0:
        raise ValueError("ValueError")

func(1,2,3)

1 2 3
ValueError ValueError


In [55]:
# Debugable Decorator

import functools

def Decorator(*ags, **kwargs):
    """Decorator docs"""
    def decorator(func):
        """decorator docs"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            """wrapper docs"""
            return func(*args, *kwargs)
        
        return wrapper

    return decorator

@Decorator()
def function(a,b,c):
    """function docs"""
    print(a,b,c)

function.__name__, function.__doc__


('function', ' function docs')

## 실전 예제

일반적인 함수에 @decorator를 통해 error handling

```python
# from web_scrapper/module/scrapper.py
# written by wonbin kim
# 2023.09.01. 14:00 Version

# function of Scrapper class  
@classmethod
def Reconnection(cls, func): # Decorator class, function
    """ Decorator for expectable error"""
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs): # augments of the function
        
        reconnection_time = kwargs.get("reconnection_time", 10)            
        try:       
            return func(*args, **kwargs)
        except (requests.exceptions.ConnectionError,
            requests.exceptions.ChunkedEncodingError,
            ConnectionResetError,
            urllib3.exceptions.ProtocolError,
            ) as e:
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{now}] at {func.__name__} ConnectionError occur.. {e}")
            time.sleep(reconnection_time)
        except AttributeError as e:
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"[{now}] at {func.__name__} AttributeError occur.. {e}")
            time.sleep(reconnection_time)  
    return wrapper
```

```python
# from web_scrapper/module/blog.py
# written by wonbin kim
# 2023.09.01. 14:00 Version

# function of Scrapper_Blog class  
@Scrapper.Reconnection
def _get_latest_text_id_brunch(self,
    target : str, # blog type
    user_id : str,
    **kwargs
):
    """Blog user id를 통해 가장 최신글 번호를 확인하는 함수"""
    url = self.BLOG[target]["list_url"].format(user_id=user_id)
    
    xtree = self.get_xtree(url, **kwargs)
    
    links = []
    
    for item in xtree.findall(".//item"):
        link = item.find("guid").text
        links.append(link)
    
    max_link = max(links)
    
    max_idx = int(max_link.split("/")[-1])
    
    if max_idx > 10000:
        raise ValueError(target, user_id, max_idx,
             "a number of post is more than 10000")
    
    return max_idx
```